In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [4]:
# get data
%run preprocess.ipynb
X_train, y_train, X_val, y_val, X_test, y_test = get_processed_data()

colums: Index(['n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'global_ra

In [5]:
# load data without preprocessing
from ucimlrepo import fetch_ucirepo

# fetch dataset
online_news_popularity = fetch_ucirepo(id=332)

# data (as pandas dataframes)
X_raw = online_news_popularity.data.features
y_raw = online_news_popularity.data.targets

# metadata
print(online_news_popularity.metadata)
# variable information
print(online_news_popularity.variables)


{'uci_id': 332, 'name': 'Online News Popularity', 'repository_url': 'https://archive.ics.uci.edu/dataset/332/online+news+popularity', 'data_url': 'https://archive.ics.uci.edu/static/public/332/data.csv', 'abstract': 'This dataset summarizes a heterogeneous set of features about articles published by Mashable in a period of two years. The goal is to predict the number of shares in social networks (popularity).', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 39797, 'num_features': 58, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': [' shares'], 'index_col': ['url'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2015, 'last_updated': 'Thu Feb 15 2024', 'dataset_doi': '10.24432/C5NS3V', 'creators': ['Kelwin Fernandes', 'Pedro Vinagre', 'Paulo Cortez', 'Pedro Sernadela'], 'intro_paper': {'title': 'A Proactive Intelligent Decision Support System for Predicti

In [6]:
# split unpreprocessed data into training, validation and test sets
X_train_raw, y_train_raw, X_val_raw, y_val_raw, X_test_raw, y_test_raw = split(X_raw, y_raw,test_size=0.2, val_size=0.3)

In [7]:
# create a random forest regressor model
rf = RandomForestRegressor(n_estimators=72, random_state=42) # n_estimators is the number of trees in the forest
rf.fit(X_train, y_train)
# create a random forest regressor model
rf_raw = RandomForestRegressor(n_estimators=72, random_state=42) # n_estimators is the number of trees in the forest
rf.fit(X_train_raw, y_train_raw)

C:\Users\rodcs\.conda\envs\data_science_tests\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\rodcs\.conda\envs\data_science_tests\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(n_estimators=72, random_state=42)

In [8]:
# Caclulate performance using the validation set
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_pred = rf.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')


C:\Users\rodcs\.conda\envs\data_science_tests\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


ValueError: X has 35 features, but RandomForestRegressor is expecting 58 features as input.

In [ ]:
# tune hyperparameters
def grid_search_tunning(n_estimators, criterion, max_depth, min_samples_split, X_train, y_train, X_val, y_val):
    best_mse = float('inf')
    best_params = {}
    for n in n_estimators:
        for c in criterion:
            for d in max_depth:
                for s in min_samples_split:
                    rf = RandomForestRegressor(n_estimators=n, criterion=c, max_depth=d, min_samples_split=s, random_state=42)
                    rf.fit(X_train, y_train)
                    y_pred = rf.predict(X_val)
                    mse = mean_squared_error(y_val, y_pred)
                    if mse < best_mse:
                        best_mse = mse
                        best_params = {'n_estimators': n, 'criterion': c, 'max_depth': d, 'min_samples_split': s}
    return best_mse, best_params

In [ ]:
grid_search_tunning(n_estimators=[100],
                    criterion=["squared_error", "friedman_mse", "absolute_error"],
                    max_depth=[None, 2, 5, 10, 20, 30, 40],
                    min_samples_split=[2, 5, 10, 20],
                    X_train=X_train, y_train=y_train,
                    X_val=X_val,
                    y_val=y_val)